### This script converts 3d gaussian ply file to have white foreground colour

In [1]:
import sys
sys.dont_write_bytecode = True

import numpy as np 
from plyfile import PlyData, PlyElement

### Constants

In [2]:
GAUSSIAN_FILE = '../gsplat_pinecone/gsplat_output/point_cloud/iteration_30000/point_cloud.ply' 
OUTPUT_FILE = '../gsplat_fortress/gsplat_output/point_cloud/iteration_30000/point_cloud.ply' 

### Modify and save to new file

In [3]:
ply = PlyData.read(GAUSSIAN_FILE)

# Access the vertex data as a numpy structured array
vertices = np.array(ply['vertex'].data)

# Modify the color properties to white
# Assuming the values are normalized between 0 and 1
vertices['f_dc_0'] = np.ones(len(vertices), dtype=np.float32)
vertices['f_dc_1'] = np.ones(len(vertices), dtype=np.float32)
vertices['f_dc_2'] = np.ones(len(vertices), dtype=np.float32)

# set all other SH params to 0 to clear view-dependent colouring
for i in range(45):
    vertices[f'f_rest_{i}'] = np.zeros(len(vertices), dtype=np.float32)

# Create a new PlyElement for the modified vertices
updated_vertices = PlyElement.describe(vertices, 'vertex')

# Save the modified PLY data back to a new file
ply = PlyData([updated_vertices], text=ply.text)
ply.write(OUTPUT_FILE)


### Render Segmented Gaussians

In [4]:
GAUSSIAN_PATH = "D:/Projects/GaussianSplat/gaussian-splatting"
CURRENT_PATH = "D:/3D Sensing/GitHubRepo/sam_gsplat" 

In [5]:
%cd {GAUSSIAN_PATH} 

D:\Projects\GaussianSplat\gaussian-splatting


c:\ProgramData\anaconda3\envs\dust3r\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!python render.py -m "D:\3D Sensing\GitHubRepo\gsplat_fortress\gsplat_output" -s "D:\3D Sensing\GitHubRepo\gsplat_fortress\gsplat_model" --data_device cuda

Looking for config file in D:\3D Sensing\GitHubRepo\gsplat_fortress\gsplat_output\cfg_args
Config file found: D:\3D Sensing\GitHubRepo\gsplat_fortress\gsplat_output\cfg_args
Rendering D:\3D Sensing\GitHubRepo\gsplat_fortress\gsplat_output
Loading trained model at iteration 30000 [26/04 20:14:41]

Reading camera 1/42
Reading camera 2/42
Reading camera 3/42
Reading camera 4/42
Reading camera 5/42
Reading camera 6/42
Reading camera 7/42
Reading camera 8/42
Reading camera 9/42
Reading camera 10/42
Reading camera 11/42
Reading camera 12/42
Reading camera 13/42
Reading camera 14/42
Reading camera 15/42
Reading camera 16/42
Reading camera 17/42
Reading camera 18/42
Reading camera 19/42
Reading camera 20/42
Reading camera 21/42
Reading camera 22/42
Reading camera 23/42
Reading camera 24/42
Reading camera 25/42
Reading camera 26/42
Reading camera 27/42
Reading camera 28/42
Reading camera 29/42
Reading camera 30/42
Reading camera 31/42
Reading camera 32/42
Reading camera 33/42
Reading camera 34/


Rendering progress: 100%|██████████| 42/42 [00:46<00:00,  1.12s/it]

Rendering progress: 0it [00:00, ?it/s]
Rendering progress: 0it [00:00, ?it/s]


In [7]:
%cd {CURRENT_PATH}

D:\3D Sensing\GitHubRepo\sam_gsplat


### Create Folder with Ground Truths and Renders

The ground truth segmentation file names are all over the place, but they do have the file number that references the same image as the render. Thus, here we are just preparing the results in a common folder such that we can easily calculate metrics on them in the main pipeline files.

In [8]:
import os  
import shutil 
import cv2
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt 

SPINNERF_DATA = 'D:/3D Sensing/GitHubRepo/fortress'  
DEST_DIR = '../rendersFortress'
RENDERED_PATH = 'D:/3D Sensing/GitHubRepo/gsplat_fortress/gsplat_output/allRenders/ours_30000/renders'  

new_size = (1008, 756) 

# Create the destination directory if it doesn't exist
if not os.path.exists(DEST_DIR):
    os.makedirs(os.path.join(DEST_DIR, 'gt'))  
    os.makedirs(os.path.join(DEST_DIR, 'renders'))

for filename in os.listdir(SPINNERF_DATA):
    # Check if the file name contains 'pseudo'
    if 'pseudo' in filename:
        # Construct the full file path
        src_file = os.path.join(SPINNERF_DATA, filename)
        
        # Extract the 6 digit number after the first underscore in the file name
        new_filename = filename.split('_')[0] + '.png' 

        # Construct the destination ground file path
        dst_file = os.path.join(DEST_DIR, 'gt' , new_filename) 

        # Construct the destination render file path 
        rend_file = os.path.join(RENDERED_PATH, new_filename) 

        # Create binary mask out of the gt and renders 
        gt_img = cv2.imread(src_file)
        rend_img = cv2.imread(rend_file)   
        rend_img =  cv2.resize(rend_img, new_size, interpolation = cv2.INTER_LINEAR)

        gray_rend = cv2.cvtColor(rend_img, cv2.COLOR_BGR2GRAY)  
        _, binary_rend = cv2.threshold(gray_rend, int(0.6 * 255), 255, cv2.THRESH_BINARY) 

        gray_gt = cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY)  
        _, binary_gt = cv2.threshold(gray_gt, int(0.0 * 255), 255, cv2.THRESH_BINARY) 

        dst_file2 = os.path.join(DEST_DIR, 'renders' , new_filename)

        # Copy and rename the file
        cv2.imwrite(dst_file, binary_gt) 
        cv2.imwrite(dst_file2, binary_rend)    